## Import Libraries

In [115]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

In [117]:
! pip install pickle5

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/wiwaaw/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/home/wiwaaw/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/wiwaaw/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 35, in __init__
    parsed = _parse_requirement(requirement_string)
  File "/home/wiwaaw/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 64, in parse_requirement
    return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
  File "/home/wiwaaw/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 82, in _parse_re

In [118]:
import pickle5 as pickle

## Load & Explore Dataset

In [119]:
df = pd.read_csv('dune_2_reviews.csv')
print(df.head())

                                        Review Title Review Date  \
0                    This is what Hollywood needs!\n  2024-02-26   
1                           Long live the fighters\n  2024-02-26   
2   Ladies and gentleman.. the PEAK of filmmaking...  2024-02-28   
3                    WOW! I need Dune Messiah now.\n  2024-02-26   
4        The Sci-Fi/Fantasy Epic of our Generation\n  2024-02-26   

                                      Review Content  User Rating  
0  This is what Hollywood needs. A great story wi...         10.0  
1  Phenomenal stuff. I'll probably calm down tomo...          9.0  
2  This is the kind of movie that is impossible t...         10.0  
3  If you liked or loved the first one, the same ...         10.0  
4  Had the pleasure to watch this film in an earl...         10.0  


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1066 entries, 0 to 1065
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Review Title    1066 non-null   object 
 1   Review Date     1066 non-null   object 
 2   Review Content  1066 non-null   object 
 3   User Rating     1056 non-null   float64
dtypes: float64(1), object(3)
memory usage: 33.4+ KB


In [121]:
print(df.isnull().sum())

Review Title       0
Review Date        0
Review Content     0
User Rating       10
dtype: int64


In [122]:
print(df['User Rating'].describe())

count    1056.000000
mean        8.579545
std         2.148805
min         1.000000
25%         8.000000
50%        10.000000
75%        10.000000
max        10.000000
Name: User Rating, dtype: float64


In [123]:
df.head()

,Review Title,Review Date,Review Content,User Rating
0,This is what Hollywood needs!\n,2024-02-26,This is what Hollywood needs. A great story wi...,10.0
1,Long live the fighters\n,2024-02-26,Phenomenal stuff. I'll probably calm down tomo...,9.0
2,Ladies and gentleman.. the PEAK of filmmaking...,2024-02-28,This is the kind of movie that is impossible t...,10.0
3,WOW! I need Dune Messiah now.\n,2024-02-26,"If you liked or loved the first one, the same ...",10.0
4,The Sci-Fi/Fantasy Epic of our Generation\n,2024-02-26,Had the pleasure to watch this film in an earl...,10.0


## Data PreProcessing & Cleaning

In [124]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [125]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Join the tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)
    
    return cleaned_text

In [126]:
df['Review Content'] = df['Review Content'].apply(clean_text)
df['Review Title'] = df['Review Title'].apply(clean_text)

In [127]:
df.head()

,Review Title,Review Date,Review Content,User Rating
0,hollywood needs,2024-02-26,hollywood needs great story great directorprod...,10.0
1,long live fighters,2024-02-26,phenomenal stuff ill probably calm tomorrow ri...,9.0
2,ladies gentleman peak filmmaking,2024-02-28,kind movie impossible justice talking kind exp...,10.0
3,wow need dune messiah,2024-02-26,liked loved first one apply one personally lov...,10.0
4,scififantasy epic generation,2024-02-26,pleasure watch film early screening completely...,10.0


In [128]:
df.rename(columns={'Review Title': 'title', 'Review Date': 'date', 'Review Content': 'review', 'User Rating': 'rating'}, inplace=True)

In [129]:
df.head()

,title,date,review,rating
0,hollywood needs,2024-02-26,hollywood needs great story great directorprod...,10.0
1,long live fighters,2024-02-26,phenomenal stuff ill probably calm tomorrow ri...,9.0
2,ladies gentleman peak filmmaking,2024-02-28,kind movie impossible justice talking kind exp...,10.0
3,wow need dune messiah,2024-02-26,liked loved first one apply one personally lov...,10.0
4,scififantasy epic generation,2024-02-26,pleasure watch film early screening completely...,10.0


### Handle missing value, replace with mean

In [130]:
df[df['rating'].isna()]

,title,date,review,rating
55,snooze fest,2024-03-01,movie wasnt hours long say hours id probably g...,NaN
149,epic novelepic adaptation,2024-03-05,since first part dune paul timothee chalamet m...,NaN
384,cant fiction wont fool us,2024-02-28,first let tell walked cinema thinking watched ...,NaN
403,villeneuves offering solidifies great piece sc...,2024-03-03,spurred revenge paul embraces fremen ways must...,NaN
544,ruin,2024-03-04,ruin suspect better ruin one unfortunately ima...,NaN
558,great movie better first one serie still bette...,2024-03-03,includes spoilers want read booksthis film rem...,NaN
573,questionable casting decisions,2024-03-03,find perplexed casting choices film particular...,NaN
667,dune part two boring way long,2024-02-28,welljust got see dune part two beautiful movie...,NaN
771,people love nowadays,2024-03-04,considered masterpiece many people direction v...,NaN
903,scifi doesnt watch scifi,2024-02-29,wife isnt stretch imagination scifi fan loved ...,NaN


In [131]:
def replace_nan(value):
    if pd.isnull(value):
        return df['rating'].mean()
    else:
        return value

df['rating'] = df['rating'].apply(replace_nan)

In [132]:
df = df[['review', 'rating']]
df['sentiment'] = df['rating'].apply(lambda x: 'positive' if x > 5 
                                    else 'negative' if x < 5
                                    else 'neutral')
df = df[['review', 'rating', 'sentiment']]
df = df.sample(frac=1).reset_index(drop=True)

In [133]:
df.head()

,review,rating,sentiment
0,following events first dune film paul atreides...,10.0,positive
1,review dune part two cinemas rating checked ra...,10.0,positive
2,second movie dune epic actors costumes scenes ...,10.0,positive
3,awesome stunning film follows spirit books eve...,10.0,positive
4,masterpiece need watch carefully every scene d...,10.0,positive


## Vectorize Review
- Transform the Review text into a series of numbers using the tokenizer. This process establishes a list of the distinct words found in the Review text and assigns a distinct integer value to each word. Utilize the pad_sequences method from Keras to guarantee uniform length for all review sequences.
- The Tokenizer provided by Keras is a convenient way to convert text data into sequences of integers. It performs tokenization, which splits the text into words, and then assigns a unique integer to each word. This process is essential for neural network models to process text data.

In [134]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['review'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['review'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

The `oov_token='<OOV>'` parameter specifies that any out-of-vocabulary words encountered during tokenization should be replaced with the <OOV> token. This helps handle unseen words during model inference or when processing new data that may contain words not seen during training.

In [135]:
sentiment_labels = pd.get_dummies(df['sentiment']).values

In [136]:
df['sentiment'].value_counts()

positive    963
negative     71
neutral      32
Name: sentiment, dtype: int64

In [137]:
sentiment_neutral = df.loc[df["sentiment"] == "neutral"]

In [171]:
sentiment_neutral.head()

,review,rating,sentiment
29,dune part two unfortunately nothing aboveavera...,5.0,neutral
60,went blind aware hype praise giving fair chanc...,5.0,neutral
88,many times havent seen movies people try build...,5.0,neutral
128,sure cinematography wonderful sound breathtaki...,5.0,neutral
147,shame first movie accurate loyal book except h...,5.0,neutral


## Split Dataset
Splitting the Dataset Into Training and Testing Sets

Use scikit-learn to randomly split the dataset into training and testing sets. The training set is to train the model to classify the sentiments of the reviews. The test set is to access how good the model is at classifying new unseen reviews.

The dataset split size is 0.2. This means that 80% of the data will train the model. And the rest 20% will test the model's performance.

In [139]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2)

Create a neural network with six layers.

- **First layer**: Embedding layer. This layer learns a dense representation of words in the vocabulary.
- **Second layer**: Conv1D layer with 64 filters and a kernel size of 5. This layer performs convolution operations on the input sequences, using a small sliding window of size 5.
- **Third layer**: GlobalMaxPooling1D. Reduces the sequence of feature maps to a single vector. It takes the maximum value over each feature map.
- **Fourth layer**: Dense. Performs a linear transformation on the input vector. 
- **Fifth layer**: Dropout. Randomly sets a fraction of the input units to 0 during training. This helps prevent overfitting.
- **Sixth layer**: Output layer. Converts the output to a probability distribution over the three possible classes: positive, neutral, and negative.

In [140]:
model = Sequential()
model.add(Embedding(5000, 100, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          500000    
                                                                 
 conv1d_3 (Conv1D)           (None, 96, 64)            32064     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                 99        
                                                      

## Training Phase

In [141]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10


27/27 [==============================] - 1s 16ms/step - loss: 0.7781 - accuracy: 0.8462 - val_loss: 0.4825 - val_accuracy: 0.8972
Epoch 2/10
27/27 [==============================] - 0s 12ms/step - loss: 0.4570 - accuracy: 0.9049 - val_loss: 0.4140 - val_accuracy: 0.8972
Epoch 3/10
27/27 [==============================] - 0s 12ms/step - loss: 0.3873 - accuracy: 0.9049 - val_loss: 0.4065 - val_accuracy: 0.8972
Epoch 4/10
27/27 [==============================] - 0s 12ms/step - loss: 0.3726 - accuracy: 0.9049 - val_loss: 0.4007 - val_accuracy: 0.8972
Epoch 5/10
27/27 [==============================] - 0s 13ms/step - loss: 0.3235 - accuracy: 0.9049 - val_loss: 0.3947 - val_accuracy: 0.8972
Epoch 6/10
27/27 [==============================] - 0s 12ms/step - loss: 0.2835 - accuracy: 0.9061 - val_loss: 0.3891 - val_accuracy: 0.8972
Epoch 7/10
27/27 [==============================] - 0s 12ms/step - loss: 0.2229 - accuracy: 0.9108 - val_loss: 0.3804 - val_accuracy: 0.8972
Epoch 8/10
27/27 [======

## Evaluating Model Performance

To calculate the accuracy of the sentiment predictor, we would compare the predicted sentiment labels `y_pred` with the actual sentiment labels `y_test`. Since this is a multi-class classification problem, we can use the `accuracy_score` function from scikit-learn's metrics module.

Use the `model.predict()` method to predict the sentiment labels for the test set. Calculate the accuracy score using the `accuracy_score()` function from scikit-learn.

In [142]:
y_pred = np.argmax(model.predict(x_test), axis=-1)
print("Accuracy:", accuracy_score(np.argmax(y_test, axis=-1), y_pred))

Accuracy: 0.897196261682243


## Saving Model
Save the model using the `model.save()` method. Use pickle to serialize and save the tokenizer object.

In [143]:
model.save('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Load Model

In [144]:
import keras

model = keras.models.load_model('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [167]:
def predict_sentiment(text):
    # Tokenize and pad the input text
    text_sequence = tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=100)

    # Make a prediction using the trained model
    predicted_rating = model.predict(text_sequence)[0]
    print(predicted_rating[2])
    if np.argmax(predicted_rating) == 0:
        return 'Negative'
    elif np.argmax(predicted_rating) == 1:
        return 'Neutral'
    else:
        return 'Positive'

## Predict your own review

In [169]:
text_input = "This movie is great, I loved it"
predicted_sentiment = predict_sentiment(text_input)
print(predicted_sentiment)

0.6532788
Positive


## Inference Time Speed

In [151]:
import time

# Example text for prediction
example_text = "This movie is great, I loved it!"

# Tokenize and pad the input text
text_sequence = tokenizer.texts_to_sequences([example_text])
text_sequence = pad_sequences(text_sequence, maxlen=100)

# Measure the time taken for prediction
start_time = time.time()
predicted_rating = model.predict(text_sequence)
end_time = time.time()

# Calculate inference speed
inference_time = end_time - start_time
print("Inference Speed (seconds):", inference_time)

Inference Speed (seconds): 0.055696725845336914


## Next Steps for Improvement

- Considering the dataset's imbalance, the next step is to address this issue by implementing various balancing techniques. The specific technique to be used is yet to be determined after further evaluation.
- Exploring classic algorithms like Support Vector Machines (SVM) or Naïve Bayes could be beneficial. These algorithms are well-established for classification tasks and may provide valuable insights alongside the Neural Network approach.
- Given that the task is defined as multiclass classification, the model's output will be a vector containing probabilities for each class. Future iterations may involve exploring binary classification for a more focused analysis.